## To load the file:
### 1. ".dcm"
    Download the folder "PAT034" which include all the slices for rendering
### 2. ".stl" and ".glb"
    I was interested to try them. these files already 3D so, don't need for volume rendering

# Code

In [1]:
from PyQt5 import uic, QtWidgets
from PyQt5.QtWidgets import QApplication, QFileDialog, QVBoxLayout
from vtk.qt.QVTKRenderWindowInteractor import QVTKRenderWindowInteractor
import vtk
import pydicom 
import numpy as np
import os

class MainWindow(QtWidgets.QMainWindow):
    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)
        
        # Set the window title
        self.setWindowTitle("Volume Rendering")

        # Load the UI Page
        uic.loadUi(r'render.ui', self)
        self.load.clicked.connect(self.load_file)
        
        # Create a QVTKRenderWindowInteractor widget
        self.vtkWidget = QVTKRenderWindowInteractor(self.widget)
        vbox = QVBoxLayout()
        vbox.addWidget(self.vtkWidget)
        self.widget.setLayout(vbox)
        
        # Create a VTK Renderer
        self.renderer = vtk.vtkRenderer()
        self.vtkWidget.GetRenderWindow().AddRenderer(self.renderer)
        
        # Check if the combobox text is changed
        self.comboBox.currentIndexChanged.connect(self.combobox_changed)
        
        # Adjust iso slider
        self.iso_slider.setMinimum(0)
        self.iso_slider.setMaximum(1000)  
        self.iso_slider.setValue(200)
        
        # Check if the iso slider value is changed
        self.iso_slider.valueChanged.connect(self.update_isoValue)
        
        # Adjust lcd number
        self.lcdNumber.setDigitCount(200) 
        self.lcdNumber.display(self.iso_slider.value())
        
    

    def load_file(self):
        # Clear the renderer
        self.renderer.RemoveAllViewProps()

        filePath, _ = QFileDialog.getOpenFileName(self, "Open File",
            "",  # The initial directory. Leave as "" to start from the working directory.
            "Files (*.dcm *.stl *.glb)"
        )
        if filePath:  # Check if a file was selected
            if filePath.endswith('.dcm'):
                # Get the directory containing the DICOM files
                dicomDir = os.path.dirname(filePath)

                # Load the DICOM series
                self.reader = vtk.vtkDICOMImageReader()
                self.reader.SetDirectoryName(dicomDir)
                self.reader.Update()

                # Check if the data is 2D
                extent = self.reader.GetOutput().GetExtent()
                if extent[1] == extent[0] or extent[3] == extent[2] or extent[5] == extent[4]:
                    print("Error: The loaded DICOM data appears to be 2D. Try loading a series of DICOM files to form a 3D volume.")
                    return

                # Create an image actor
                imageActor = vtk.vtkImageActor()
                imageActor.GetMapper().SetInputConnection(self.reader.GetOutputPort())

                # Add the image actor to the renderer
                self.renderer.AddActor(imageActor)

                # Render the window
                self.vtkWidget.GetRenderWindow().Render()
                
            elif filePath.endswith('.stl'):
                # Load the STL file
                reader = vtk.vtkSTLReader()
                reader.SetFileName(filePath)
                reader.Update()

                # Create a mapper and actor for the STL data
                mapper = vtk.vtkPolyDataMapper()
                mapper.SetInputConnection(reader.GetOutputPort())

                actor = vtk.vtkActor()
                actor.SetMapper(mapper)

                # Add the actor to the renderer and render the window
                self.renderer.AddActor(actor)
                self.vtkWidget.GetRenderWindow().Render()

            elif filePath.endswith('.glb'):
                # Load the GLB file
                reader = vtk.vtkGLTFImporter()
                reader.SetFileName(filePath)
                reader.SetRenderWindow(self.vtkWidget.GetRenderWindow())
                reader.Read()

                # Update the renderer and render the window
                self.renderer.ResetCamera()
                self.vtkWidget.GetRenderWindow().Render()
                
    def combobox_changed(self):
        # Clear the renderer
        self.renderer.RemoveAllViewProps()
        
        if self.comboBox.currentText().strip() == "Surface Rendering":
            self.apply_surface_rendering()
        else:
            self.apply_ray_casting_rendering()
        print(repr(self.comboBox.currentText().strip()))
        print(repr("Surface Rendering"))
        
        # Render the window
        self.vtkWidget.GetRenderWindow().Render()
        
    def update_isoValue(self, value):
        # Update ISO value in the LCDNumber
        self.lcdNumber.display(value)

        # Apply surface rendering with the updated ISO value
        self.apply_surface_rendering()

    def apply_surface_rendering(self):
        # Clear the renderer
        self.renderer.RemoveAllViewProps()

        # Check if DICOM data is already loaded
        if not hasattr(self, 'reader'):
            print("Error: No DICOM data loaded. Load a DICOM file first.")
            return
        
        # Get ISO value from the slider
        isoValue = self.iso_slider.value()
        
        # Check if the loaded data is 2D
        extent = self.reader.GetOutput().GetExtent()
        if extent[1] == extent[0] or extent[3] == extent[2] or extent[5] == extent[4]:
            print("Error: The loaded DICOM data appears to be 2D. Try loading a series of DICOM files to form a 3D volume.")
            return

        # Create a marching cubes filter to extract surface
        marchingCubes = vtk.vtkMarchingCubes()
        marchingCubes.SetInputConnection(self.reader.GetOutputPort())
        marchingCubes.SetValue(0, isoValue)  # You need to set a threshold value

        # Create mapper and actor for the surface
        surfaceMapper = vtk.vtkPolyDataMapper()
        surfaceMapper.SetInputConnection(marchingCubes.GetOutputPort())

        surfaceActor = vtk.vtkActor()
        surfaceActor.SetMapper(surfaceMapper)

        # Add the surface actor to the renderer
        self.renderer.AddActor(surfaceActor)

        # Render the window
        self.vtkWidget.GetRenderWindow().Render()
        
    def apply_ray_casting_rendering(self):
        # Clear the renderer
        self.renderer.RemoveAllViewProps()

        # Check if DICOM data is already loaded
        if not hasattr(self, 'reader'):
            print("Error: No DICOM data loaded. Load a DICOM file first.")
            return

        # Check if the loaded data is 2D
        extent = self.reader.GetOutput().GetExtent()
        if extent[1] == extent[0] or extent[3] == extent[2] or extent[5] == extent[4]:
            print("Error: The loaded DICOM data appears to be 2D. Try loading a series of DICOM files to form a 3D volume.")
            return

        # Create a volume mapper for ray casting rendering
        volumeMapper = vtk.vtkSmartVolumeMapper()
        volumeMapper.SetBlendModeToComposite()

        # Create a volume property
        volumeProperty = vtk.vtkVolumeProperty()
        volumeProperty.ShadeOn()
        volumeProperty.SetInterpolationTypeToLinear()

        # Create a color transfer function
        colorTransferFunction = vtk.vtkColorTransferFunction()
        colorTransferFunction.AddRGBPoint(0, 0.0, 0.0, 1.0)  # Blue at scalar value 0
        colorTransferFunction.AddRGBPoint(200, 1.0, 1.0, 1.0)  # White at scalar value 200
        colorTransferFunction.AddRGBPoint(1000, 1.0, 0.0, 0.0)  # Red at scalar value 1000

        # Set the color transfer function to the volume property
        volumeProperty.SetColor(colorTransferFunction)

        # Set opacity to 0.5 (semi-transparent)
        opacityFunction = vtk.vtkPiecewiseFunction()
        opacityFunction.AddPoint(0, 0.0)
        opacityFunction.AddPoint(255, 0.5)  # Adjust the values as needed
        volumeProperty.SetScalarOpacity(opacityFunction)

        # Create a volume
        volume = vtk.vtkVolume()
        volume.SetMapper(volumeMapper)
        volume.SetProperty(volumeProperty)

        # Get the output of the reader
        image_data = self.reader.GetOutput()

        # Set the input data for the volume mapper
        volumeMapper.SetInputData(image_data)

        # Set the volume to the renderer
        self.renderer.AddVolume(volume)

        # Render the window
        self.vtkWidget.GetRenderWindow().Render()


    


if __name__ == "__main__":
    import sys

    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec_())


'Surface Rendering'
'Surface Rendering'
'Ray Casting Rendering'
'Surface Rendering'


SystemExit: 0

F:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
